**note:** this is just a notebook for hyperparameter optimisation experiments; the only reason i've pushed it is so it can be used with Colab.

In [1]:
%load_ext autoreload
%autoreload 2

from pykeen.hpo import hpo_pipeline, hpo_pipeline_from_config

import sys
sys.path.append("..")

from src.cli.utils import triplesfactory_from_tsv

In [2]:
tf = triplesfactory_from_tsv("../data/interim/triples_filtered_by_predicate_small.csv")
training, testing, validation = tf.split([.8, .1, .1], random_state=100)

In [14]:
hpo_config = {
    "pipeline": dict (
        model_kwargs_ranges = dict( 
            embedding_dim=dict(type=int, low=64, high=512, step=32) 
        ),
        negative_sampler_kwargs_ranges=dict(
            num_negs_per_pos=dict(type=int, low=10, high=100, log=True),
        ),
        optimizer_kwargs_ranges=dict(
            lr=dict(type=float, low=0.001, high=0.1, log=True)
        )
        ,
        training_kwargs_ranges=dict(
#             num_epochs=dict(type=int, low=100, high=1000, step=300, log=False)
            num_epochs=dict(type=int, low=1, high=3, step=2, log=False)
        )
    ),
    "optuna": dict(
        n_trials=2 # can use timeout or n_trials here
    ),
}

hpo_pipeline_from_config(
    config=hpo_config,
    training=training,
    testing=testing,
    validation=validation,
    model="TransE",
    device="cpu",
)

[I 2021-07-05 10:38:19,145] A new study created in memory with name: no-name-2fb43bb9-6505-4508-91de-39b8d2993d52
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.trans_e.TransE'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.MarginRankingLoss'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize adjusted_arithmetic_mean_rank_index
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True


Training epochs on cpu:   0%|          | 0/3 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/230 [00:00<?, ?batch/s]

KeyboardInterrupt: 